### BIM to BuildingSceneLayer

#### This Notebook imports revit files from BIM models and convert them to BuildingSceneLayer in prepartion for smart map
#### Please implement the following steps in preparing building layers to be published to Portal for ArcGIS.
1. Define Projection to PRS92
2. BIM File to Geodatabase
3. Make Building Layer
4. Enable Archiving, GlobalIDs, and Editor Tracking
5. Add Fields for Structural discipline layers
6. Apply symbology to Status
7. Set the initial status = 'To be Constructed'
8. Add station names and CP
9.Assign Domains


In [2]:
import time, os, datetime, sys, logging, logging.handlers, shutil, traceback, re
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("WGS 1984 Web Mercator (auxiliary sphere)")
arcpy.env.geographicTransformations = "PRS_1992_To_WGS_1984_1"
workspace = r'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2'
#workspace = r'C:/Users/oc3512/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2/During-Construction_nscrexn2.gdb'

BuldingLayerName = "N2_Station_Structure"

In [ ]:
# 1. Define Projection
## Manually check projection of all Revit models of structural disciplines
## Make sure that all the models have projection defined (i.e., at the right location)

In [3]:
out_gdb_path = os.path.join(workspace,"10.10.5.5.sde")
out_dataset_name = "N2_Station_Structure_BIM"

In [6]:
# 2. BIM File to Geodatabase
## This saves all Revit models to enterprise geodatabase
RevitWD = "C:/Users/eiji.LAPTOP-KSD9P6CP/Dropbox/01-Railway/02-NSCR-Ex/01-N2/01-Basedata/09-Station_Structure/99-BIM Model/"
Revit1 = os.path.join(RevitWD,"N-01/NSCR-GCR-N01-APLSTN-MOD-ST-000001.rvt")
Revit2 = os.path.join(RevitWD, "N-01/NSCR-GCR-N01-CLPSTN-MOD-ST-000001.rvt")
Revit3 = os.path.join(RevitWD, "N-02/NSCR-GCR-N02-CSFSTN-MOD-ST-000001.rvt")
Revit4 = os.path.join(RevitWD, "N-03/NSCR-GCR-N03-ANGSTN-MOD-ST-000001.rvt")
Revit5 = os.path.join(RevitWD, "N-03/NSCR-GCR-N03-CRKSTN-MOD-ST-000001.rvt")
Revit6 = os.path.join(RevitWD, "N-04/NSCR-GCR-N04-CIASTN-MOD-ST-000001.rvt")

inputRevit = [Revit1,Revit2,Revit3,Revit4,Revit5,Revit6]
# Choose enterprise geodatabase for Output Geodatabase
spatial_reference = "PRS_1992_Philippines_Zone_III"

arcpy.BIMFileToGeodatabase_conversion(inputRevit, out_gdb_path, out_dataset_name, spatial_reference)


<Result 'C:/Users/eiji.LAPTOP-KSD9P6CP/OneDrive - Oriental Consultants Global JV/Documents/ArcGIS/Projects/During-Construction_nscrexn2\\10.10.5.5.sde\\NSCR_Ex.NSCREXUSER.N2_Station_Structure_BIM'>

In [ ]:
# 3. Make Building Layer
## Convert files created above to building layer
# Make a building layer from a Dataset
# The code below failed. why? but succeeded when it was manually executed in Geoprocessing.
out_dataset = os.path.join(out_gdb_path,"NSCR_Ex.NSCREXUSER.N2_Station_Structure_BIM")
arcpy.MakeBuildingLayer_management(out_dataset,BuldingLayerName)

In [10]:
# 4. Enable Archiving, GlobalIDs, and Editor Tracking by
#    right-clicking enterprise geodatabase created by 'BIM File to Geodatabase' tool

In [4]:
stColumn = os.path.join(BuldingLayerName,"Structural","StructuralColumns")
stFraming = os.path.join(BuldingLayerName,"Structural","StructuralFraming")
stFoundation = os.path.join(BuldingLayerName,"Structural","StructuralFoundation")

in_features = [stColumn, stFraming, stFoundation]
addFields = ['Station', 'Types', 'SubType', 'Status', 'start_date', 'end_date', \
             'StructureType', 'StructureLevel', 'P6ID', 'Comment', \
            'GridX', 'GridY', 'CP', 'ID']

In [16]:
# 5. Add Fields for Structural discipline layers
## StructuralColumns, StructuralFraming, and StructuralFoundation
for feature in in_features:
    for field in addFields:
        if field in ('start_date', 'end_date'):
            arcpy.AddField_management(feature, field, "DATE", "", "", "", field, "NULLABLE", "")
        elif field in ('P6ID', 'Comment', 'GridX', 'GridY', 'CP', 'ID'):
            arcpy.AddField_management(feature, field, "TEXT", "", "", "", field, "NULLABLE", "")
        else:
            arcpy.AddField_management(feature, field, "SHORT", "", "", "", field, "NULLABLE", "")

In [17]:
# 6. Apply symbology to Status
symbolLyrx = r'C:/Users/eiji.LAPTOP-KSD9P6CP/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structure_FINAL.lyrx'

for feature in in_features:
    arcpy.ApplySymbologyFromLayer_management(feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]

In [18]:
# 7. Set the initial status = 'To be Constructed'
for feature in in_features:
    with arcpy.da.UpdateCursor(feature, ['Status']) as cursor:
        for row in cursor:
            row[0] = 1
            cursor.updateRow(row)

In [31]:
# 8. Add station names and CP
searchStns = "APLSTN|CLPSTN|CSFSTN|ANGSTN|CRKSTN|CIASTN"

# Enter 'Station'
for feature in in_features:
    with arcpy.da.UpdateCursor(feature,['DocName','Station','CP']) as cursor:
        for row in cursor:
            if row[0]:
                try:
                    reg = re.search(searchStns,row[0]).group()
                except AttributeError:
                    reg = re.search(searchStns,row[0])
                
                # Station name
                if reg == "APLSTN":
                    row[1] = 7
                    row[2] = "N-01"
                elif reg == "CLPSTN":
                    row[1] = 8
                    row[2] = "N-01"
                elif reg == "CSFSTN":
                    row[1] = 6
                    row[2] = "N-02"
                elif reg == "ANGSTN":
                    row[1] = 5
                    row[2] = "N-03"
                elif reg == "CRKSTN":
                    row[1] = 4
                    row[2] = "N-03"
                elif reg == "CIASTN":
                    row[1] = 3
                    row[2] = "N-04"
            else:
                row[1] = None
                row[2] = None
            cursor.updateRow(row)

In [32]:
# Enter CP
for feature in in_features:
    with arcpy.da.UpdateCursor(feature,['DocName','CP']) as cursor:
        for row in cursor:
            if row[0]:
                try:
                    reg = re.search(searchStns,row[0]).group()
                except AttributeError:
                    reg = re.search(searchStns,row[0])
                
                # Station name
                if reg == "APLSTN":
                    row[1] = "N-01"
                elif reg == "CLPSTN":
                    row[1] = "N-01"
                elif reg == "CSFSTN":
                    row[1] = "N-02"
                elif reg == "ANGSTN":
                    row[1] = "N-03"
                elif reg == "CRKSTN":
                    row[1] = "N-03"
                elif reg == "CIASTN":
                    row[1] = "N-04"
            else:
                row[1] = None
            cursor.updateRow(row)

In [5]:
# 9.Assign Domains
# 0. Domain Name
domainList = ['Station Structures_TYPE', 'Station Structures_SUBTYPE',\
              'Station Structures_STATUS', 'Station Structures_STRUCTURETYPE',\
              'Station Structures_STRUCTURELEVEL','Station_nscrex']

#domains = arcpy.da.ListDomains(workspace)
#listDomain = [d.name for d in domains]
#reg = re.compile(r'Station Structures*|Station*')
#listFinal = list(filter(reg.match, listDomain))
for feature in in_features:
    arcpy.AssignDomainToField_management(feature, 'Types', domainList[0])
    arcpy.AssignDomainToField_management(feature, 'SubType', domainList[1])
    arcpy.AssignDomainToField_management(feature, 'Status', domainList[2])
    arcpy.AssignDomainToField_management(feature, 'StructureType', domainList[3])
    arcpy.AssignDomainToField_management(feature, 'StructureLevel', domainList[4])
    arcpy.AssignDomainToField_management(feature, 'Station', domainList[5])

## Replace Pile and Pile Caps

#### 1. Delete pile/pile caps from StructuralFoundation (As pile and pile caps in BIM models are groupd together, we need to replace them with ours.)

#### 2. 'Make Building Layer' again
#### 3. Re-apply symbology
#### 4. Add necessary information
#### 5. Append pile/pile caps to StructuralFoundation
#### 6. 'Make Building Layer' again

In [5]:
# 1. Delete piles/pile caps from StructuralFoundation
## Note that when run the following and try to delete rows, this edit is not reflected in the 'Make Building Layer'.
## But the change is reflected in the source layer in the enterprise geodatabase.

with arcpy.da.UpdateCursor(stFoundation,["OmniClassDescription"]) as cursor:
    for row in cursor:
        if row[0] == "Foundation Piles":
            cursor.deleteRow()

## You need to run 'Make Building Layer' again due to the above reason.

In [4]:
# 2. 'Make Building Layer' again
out_gdb_path = os.path.join(workspace,"10.10.5.5.sde")
out_dataset = os.path.join(out_gdb_path,"NSCR_Ex.NSCREXUSER.N2_Station_Structure_BIM")

arcpy.MakeBuildingLayer_management(out_dataset,BuldingLayerName)

<Result 'N2_Station_Structure'>

In [1]:
# 3. Add necessary information to pile/pile caps layer before appending to Building Layer
### I created empty multipatch layer, imported field names from the Make Building Layer, and
### appended separated pile and pile caps to the empty layer. The layer is named "N2_Pile_PileCaps"

## BaseCategory = StructuralFoundation
## Discipline = Structural
## AssemblyDesc = Pile Caps
## BldgLevel = 1 (00_TOP OF PILE)
## Category = Structural Foundations
## OmniClassDescription = Foundation Piles
## BldgLevel_Desc = 00_TOP OF PILE
## TopLevel = 00_TOP OF PILE
## TopLevel_Desc = 00_TOP OF PILE
## CreatedPhase = 2

## *********** VERY IMPORTNAT ***********
# If CreatedPhase field is empty, nothing will be displayed. Make sure to fill CreatedPhase field.

input_feature = "N2_Pile_PileCaps"

with arcpy.da.UpdateCursor(input_feature,["BaseCategory","Discipline","AssemblyDesc","BldgLevel","Category","OmniClassDescription","BldgLevel_Desc","TopLevel","TopLevel_Desc","CreatedPhase"]) as cursor:
    for row in cursor:
        row[0] = "StructuralFoundation"
        row[1] = "Structural"
        row[2] = "Pile Caps"
        row[3] = 1
        row[4] = "Structural Foundations"
        row[5] = "Foundation Piles"
        row[6] = "00_TOP OF PILE"
        row[7] = 1
        row[8] = "00_TOP OF PILE"
        row[9] = 2
        cursor.updateRow(row)


In [11]:
# 4. Add pile and pile caps to StructuralFoundation
## In other words, new pile and pile caps that are separated by the GIS Team
## Check the coordinate system
source_ref = arcpy.Describe(input_feature).spatialReference.name
target_ref = arcpy.Describe(stFoundation).spatialReference.name

input_feature = "N2_Pile_PileCaps"

if source_ref == target_ref:
    arcpy.Append_management(input_feature, stFoundation, schema_type = 'NO_TEST')
else:
    print("Two Layers have different coordinate system. PLEASE CHECK")

<Result 'N2_Station_Structure\\Structural\\StructuralFoundation'>

In [19]:
# 5. 'Make Building Layer' again
out_gdb_path = os.path.join(workspace,"10.10.5.5.sde")
out_dataset = os.path.join(out_gdb_path,"NSCR_Ex.NSCREXUSER.N2_Station_Structure_BIM")

arcpy.MakeBuildingLayer_management(out_dataset,BuldingLayerName)

<Result 'N2_Station_Structure'>

## Join Previously Worked Layer to Building Layer

### Note that we entered lots of information to previously worked station-structure layer.
### Now that we use Building Layer, we need to join information to this Building Layer

#### 1. Delete fields from building layers
#### 2. Join 

In [7]:
# 1. Delete fiels from Building Layer
## Field Names to be deleted: Status, P6ID, StructureType, StructureLevel, Types, SubType,
## Comment, GridX, GridY, uniqueID

fields_delete = ['Types','SubType','Status','StructureLevel','P6ID','Comment','StructureType',
                'GridX', 'GridY', 'uniqueID']

for feature in in_features:
    arcpy.DeleteField_management(feature, fields_delete)     

In [10]:
# 2. Join Fields
inputTable = "N2_Station_Structure_local copy"

inputJoinField = "ObjectId"
joinTableField = "ObjectId1"
transferFields = fields_delete

for feature in in_features:
    arcpy.JoinField_management(in_data=feature, in_field=inputJoinField, join_table=inputTable,
                               join_field=joinTableField, fields=transferFields)


In [11]:
# 6. Apply symbology again
symbolLyrx = r'C:/Users/eiji.LAPTOP-KSD9P6CP/Dropbox/01-Railway/02-NSCR-Ex/10-ArcGIS/01-Reference/01-Layer File/03-Station_Structure/Station_Structure_FINAL.lyrx'

for feature in in_features:
    arcpy.ApplySymbologyFromLayer_management(feature, symbolLyrx, [["VALUE_FIELD", "Status", "Status"]], update_symbology="MAINTAIN")[0]